In [79]:
import numpy as np
import cv2 as cv
from glob import glob 
import os
import json
from tqdm.notebook import tqdm

In [91]:
def parse_kitti_calib(calib):
    data = dict({'calib_cam_to_cam':{}, 'velo_to_cam':{}, 'imu_to_velo':{}})
    with open(calib) as f:
        lines = f.readlines()
    for line in lines:
        if len(line.split(':'))>1:
            key, value_str = line.split(':')
            values = value_str.split(' ')[1:]
            values = [float(x) for x in values]
            if key.startswith('P'):
                data['calib_cam_to_cam'][key+'_rect'] = np.resize(values, (3, 4))
            elif key=='R0_rect':
                data['calib_cam_to_cam'][key] = np.resize(values, (3, 3))
            elif key.startswith('Tr'):
                upper_key = key[3:]
                data[upper_key]['T'] = np.resize(values[:3], (3, 1))
                data[upper_key]['r'] = np.resize(values[3:], (3, 3))
    return data

def kitti_to_nusence(kitti_calib):
    nuscence_calib = {}
    r0_rect = np.zeros((4, 4))
    r0_rect[:3, :3] = kitti_calib['calib_cam_to_cam']['R0_rect']
    r0_rect[3, 3] = 1
    for key in kitti_calib['calib_cam_to_cam'].keys():
        if key.startswith('P'):
            Q = kitti_calib['calib_cam_to_cam'][key] @ r0_rect
            K,_,_,_,_,_,_ = cv.decomposeProjectionMatrix(Q)
            K[abs(K)<10^-6] = 0.0
            K[2,2] = 1.0
            nuscence_calib[key[:2]] = Q
            nuscence_calib[key[:2]+'_intrinsic'] = K
    nuscence_calib['P_intrinsic'] = cv.decomposeProjectionMatrix(kitti_calib['calib_cam_to_cam']['P0_rect'])[0]
    return nuscence_calib

def extract(calib_path):
    calib_kitti = parse_kitti_calib(calib_path)
    calib_nusence = kitti_to_nusence(calib_kitti)
    img_path = calib_path.replace('calib', 'image_2').replace('.txt', '.png')
    calib = {"images":[{'file_name':img_path, 'cam_intrinsic':calib_nusence['P0_intrinsic']}]}
    return calib
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [92]:
files = glob('../../data/KITTI/training/calib/*.txt')
for f in tqdm(files):
    calib_path = f.replace('calib', 'calib_nuscene').replace('.txt', '.json')
    folder_path = '/'.join(calib_path.split('/')[:-1])
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    calib = extract(f)
    fp = open(calib_path, 'w+')
    json.dump(calib, fp, cls=NumpyEncoder)

  0%|          | 0/7481 [00:00<?, ?it/s]

In [71]:
files

[]